In [16]:
import os
import json
import gdown
import pandas as pd
import numpy as np
### Plot
import matplotlib.pyplot as plt
import seaborn as sns
### HTML
from bs4 import BeautifulSoup
import re
### Warnings
import warnings
### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

In [17]:
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') #tmdb dataset we will be working with
tmdb_df
tmdb_df.describe()


,id,vote_average,vote_count,revenue,runtime,budget,popularity
count,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06
mean,6.625906e+05,2.101624e+00,2.139664e+01,7.421149e+05,5.059751e+01,2.924065e+05,1.349021e+00
std,3.549163e+05,3.110503e+00,3.391106e+02,1.803934e+07,6.229370e+01,5.134357e+06,8.071102e+00
min,2.000000e+00,0.000000e+00,0.000000e+00,-1.200000e+01,-2.800000e+01,0.000000e+00,0.000000e+00
25%,3.705740e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e-01
50%,6.643460e+05,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0.000000e+00,6.000000e-01
75%,9.721775e+05,5.000000e+00,1.000000e+00,0.000000e+00,9.000000e+01,0.000000e+00,9.140000e-01
max,1.254286e+06,1.000000e+01,3.449500e+04,3.000000e+09,1.440000e+04,9.000000e+08,2.994357e+03


In [18]:
# in order to create a robust key "title_year to join with the movielens dataset"
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('nan', '')
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1)
tmdb_df


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili",2010,Inception (2010)
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,tt0816692,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,2014,Interstellar (2014)
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,185000000,https://www.warnerbros.com/movies/dark-knight/,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin",2008,The Dark Knight (2008)
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,237000000,https://www.avatar.com/movies/avatar,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish",2009,Avatar (2009)
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian",2012,The Avengers (2012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002262,664753,Dead Kennedys: Live in Vienna,0.000,0,Released,1982-01-01,0,16,False,NaN,0,NaN,NaN,en,Dead Kennedys: Live in Vienna,Dead Kennedys filmed live at Arena Wien in Vie...,0.600,/nQ9RmZhdYhv8QHCoHgNBfyqewB6.jpg,NaN,Music,Welder,United States of America,English,1982,Dead Kennedys: Live in Vienna (1982)
1002263,664754,Haunted House,0.000,0,Released,2019-05-31,0,53,False,NaN,0,NaN,tt10402510,en,Haunted House,Three Paranormal Investigators spend the night...,0.600,/q72Scu36kQzoZX8TQ9vxnCLqDk1.jpg,NaN,Documentary,NaN,United States of America,English,2019,Haunted House (2019)
1002264,664755,"Fuck You, Purdue",0.000,0,Released,1987-07-07,0,12,False,NaN,0,NaN,NaN,en,"Fuck You, Purdue",A 1987 video by Cecilia Dougherty,0.600,NaN,NaN,NaN,NaN,NaN,NaN,1987,"Fuck You, Purdue (1987)"
1002265,664756,These Are the Rules,0.000,0,Released,1983-03-03,0,5,False,NaN,0,NaN,NaN,en,These Are the Rules,"In These Are the Rules, Doug Hall enacted an a...",0.600,NaN,NaN,NaN,NaN,NaN,NaN,1983,These Are the Rules (1983)


In [19]:
df_movies = pd.read_csv('ml-1m/movies.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genres']) #sohai give us 3 different files so need read them separately
print(df_movies.head())

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [20]:
df_ratings = pd.read_csv('ml-1m/ratings.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp']) # ^same
print(df_ratings.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [21]:
df_users = pd.read_csv('ml-1m/users.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode']) # ^same
print(df_users.head())

   UserID Gender  Age  Occupation ZipCode
0       1      F    1          10   48067
1       2      M   56          16   70072
2       3      M   25          15   55117
3       4      M   45           7   02460
4       5      M   25          20   55455


In [22]:
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID') #this is the merged version of the MovieLens Dataset
df_ml_combined

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,6,11106
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,6,11106
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,6,11106
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,6,11106


In [23]:
#now to finally merge the datasets
merged_df = pd.merge(df_ml_combined, tmdb_df, left_on='Title', right_on='title_year', how='left') #main dataset
merged_df
print(merged_df)   #use Genres from movielens

         UserID  MovieID  ...  year                              title_year
0             1     1193  ...  1975  One Flew Over the Cuckoo's Nest (1975)
1             1      661  ...  1996        James and the Giant Peach (1996)
2             1      914  ...  1964                     My Fair Lady (1964)
3             1     3408  ...  2000                  Erin Brockovich (2000)
4             1     2355  ...   NaN                                     NaN
...         ...      ...  ...   ...                                     ...
1012483    6040     1091  ...  1989              Weekend at Bernie's (1989)
1012484    6040     1094  ...   NaN                                     NaN
1012485    6040      562  ...   NaN                                     NaN
1012486    6040     1096  ...  1982                  Sophie's Choice (1982)
1012487    6040     1097  ...  1982       E.T. the Extra-Terrestrial (1982)

[1012488 rows x 35 columns]


In [24]:
print(merged_df.columns)
# Counting how many NA values are in the 'UserID' column
user_id_na_count = merged_df['UserID'].isna().sum()

print("Number of NA values in the UserID column:", user_id_na_count)
# Assuming 'df' is your DataFrame
important_columns = ['Title', 'Genres', 'overview', 'tagline', 'original_language', 'release_date', 'runtime', 'vote_average', 'vote_count', 'production_companies', 'production_countries', 'spoken_languages', 'year']

# Check for any NA values in these columns
na_columns = merged_df[important_columns].isna().any()

# Print columns with NA values
print("Columns with NA values:", na_columns[na_columns].index.tolist())



Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Title', 'Genres', 'Gender',
       'Age', 'Occupation', 'ZipCode', 'id', 'title', 'vote_average',
       'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult',
       'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path', 'tagline',
       'genres', 'production_companies', 'production_countries',
       'spoken_languages', 'year', 'title_year'],
      dtype='object')
Number of NA values in the UserID column: 0
Columns with NA values: ['overview', 'tagline', 'original_language', 'release_date', 'runtime', 'vote_average', 'vote_count', 'production_companies', 'production_countries', 'spoken_languages', 'year']


In [25]:
df = pd.read_csv('Merged_df.csv')
df.describe()
print(df.columns)



C:\Users\cindy\AppData\Local\Temp\ipykernel_7268\221871889.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Merged_df.csv')


Index(['Unnamed: 0', 'UserID', 'MovieID', 'Rating', 'Timestamp', 'Title',
       'Genres', 'Gender', 'Age', 'Occupation', 'ZipCode', 'Movie_Title',
       'Movie_Year', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path',
       'budget', 'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'year', 'title_year'],
      dtype='object')


In [26]:
# Assuming 'merged_df' is your DataFrame
merged_df = df
content_merged_df = merged_df.drop(['genres', 'poster_path', 'backdrop_path','Age', 'Gender', 'homepage','Occupation', 'ZipCode'], axis=1)
content_merged_df = content_merged_df.drop_duplicates().reset_index(drop=True) #no duplicates since row numbers remained the same, timestamp used for temporal analysis
print(content_merged_df)
#content_merged_df.to_csv('content.csv', index=False)

content_merged_df['Genres'] = content_merged_df['Genres'].apply(lambda x: x.split('|') if pd.notnull(x) else []) # make genres a list of strings

        Unnamed: 0  UserID  ...    year                              title_year
0                0       1  ...  1975.0  One Flew Over the Cuckoo's Nest (1975)
1                1       1  ...  1996.0        James and the Giant Peach (1996)
2                2       1  ...  1964.0                     My Fair Lady (1964)
3                3       1  ...  2000.0                  Erin Brockovich (2000)
4                4       1  ...  1998.0                     A Bug's Life (1998)
...            ...     ...  ...     ...                                     ...
995650      994980    6040  ...  1992.0                  The Crying Game (1992)
995651      994981    6040  ...  1992.0                  The Crying Game (1992)
995652      994982    6040  ...  1996.0         Welcome to the Dollhouse (1996)
995653      994983    6040  ...  1982.0                  Sophie's Choice (1982)
995654      994984    6040  ...  1982.0       E.T. the Extra-Terrestrial (1982)

[995655 rows x 30 columns]


In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Model

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, GlobalAveragePooling1D


# Example DataFrame columns: 'MovieID', 'Title', 'Genres', 'Overview', ...

# Preprocess text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")

# Fill NaN values with an empty string or some placeholder text before tokenization
content_merged_df['overview'].fillna("", inplace=True)

# Now proceed with the text tokenization
tokenizer.fit_on_texts(content_merged_df['overview'])
overview_sequences = tokenizer.texts_to_sequences(content_merged_df['overview'])
overview_padded = pad_sequences(overview_sequences, maxlen=100)


# Preprocess genres
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(content_merged_df['Genres'])

# Text input branch
text_input = Input(shape=(100,), dtype='int32', name='text_input')  # This line is correct as is
text_embedding = Embedding(input_dim=10000, output_dim=64)(text_input)  # Removed the incorrect argument
text_pooled = GlobalAveragePooling1D()(text_embedding)


# Genre input branch
genre_input = Input(shape=(genres_encoded.shape[1],), name='genre_input')
genre_dense = Dense(64, activation='relu')(genre_input)

# Combine branches
combined = concatenate([text_pooled, genre_dense])
combined_dense = Dense(128, activation='relu')(combined)
output = Dense(genres_encoded.shape[1], activation='softmax')(combined_dense)

# Final model
model = Model(inputs=[text_input, genre_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# overview_padded and genres_encoded need to be split separately
X_train_overview, X_test_overview, y_train_genres, y_test_genres = train_test_split(
    overview_padded, genres_encoded, test_size=0.2, random_state=42)

# The model expects a list of numpy arrays as input, so we need to split each input array separately
X_train_genre = genres_encoded[:len(X_train_overview)]
X_test_genre = genres_encoded[len(X_train_overview):]

# Train the model. Notice that model.fit expects a list of inputs for the model's multiple inputs
model.fit([X_train_overview, X_train_genre], y_train_genres, epochs=5, validation_data=([X_test_overview, X_test_genre], y_test_genres))

# Make sure to use the right layer name that you've defined for your embeddings
embedding_layer_name = 'combined_dense'
embedding_model = Model(inputs=model.input, outputs=model.get_layer(embedding_layer_name).output)

# Predict to get the embeddings
movie_embeddings = embedding_model.predict([overview_padded, genres_encoded])

# embedding_model = Model(inputs=model.input,
#                         outputs=model.get_layer('combined_dense').output)
# movie_embeddings = embedding_model.predict([overview_padded, genres_encoded])

from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(movie_id, top_n=5):
    # Assuming the MovieID is the index in content_merged_df
    movie_idx = content_merged_df.index[content_merged_df['MovieID'] == movie_id][0]
    movie_embedding = movie_embeddings[movie_idx].reshape(1, -1)
    cos_similarities = cosine_similarity(movie_embedding, movie_embeddings)[0]
    
    # Get the top_n most similar movies
    similar_movie_idxs = np.argsort(cos_similarities)[-top_n-1:-1][::-1]
    
    # Return their titles
    return content_merged_df.iloc[similar_movie_idxs]['Title']

# Example usage
given_movie_id = 1193  # Replace with an actual MovieID from your dataset
recommended_titles = recommend_movies(given_movie_id)
print("Recommended Movies:", recommended_titles.tolist())


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 100, 64)   │    640,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ genre_input         │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ embedding_2[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      1,216 │ genre_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     16,512 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 18)        │      2,322 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 660,050 (2.52 MB)

 Trainable params: 660,050 (2.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
24892/24892 ━━━━━━━━━━━━━━━━━━━━ 141s 6ms/step - accuracy: 0.1532 - loss: 40136.3242 - val_accuracy: 0.2095 - val_loss: 268520.9062
Epoch 2/5
24892/24892 ━━━━━━━━━━━━━━━━━━━━ 145s 6ms/step - accuracy: 0.1507 - loss: 403722.2188 - val_accuracy: 0.0383 - val_loss: 861721.2500
Epoch 3/5
24892/24892 ━━━━━━━━━━━━━━━━━━━━ 135s 5ms/step - accuracy: 0.1515 - loss: 1108906.2500 - val_accuracy: 0.2557 - val_loss: 1430489.5000
Epoch 4/5
24892/24892 ━━━━━━━━━━━━━━━━━━━━ 130s 5ms/step - accuracy: 0.1515 - loss: 2142906.0000 - val_accuracy: 0.0177 - val_loss: 4458455.5000
Epoch 5/5
24892/24892 ━━━━━━━━━━━━━━━━━━━━ 129s 5ms/step - accuracy: 0.1518 - loss: 3577516.7500 - val_accuracy: 0.2557 - val_loss: 4516542.0000


ValueError: No such layer: combined_dense. Existing layers are: ['text_input', 'embedding_2', 'genre_input', 'global_average_pooling1d_1', 'dense_3', 'concatenate_1', 'dense_4', 'dense_5'].

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Define the CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the 'Genres' column
genres = vectorizer.fit_transform(content_merged_df['Genres']).toarray()

# Create a DataFrame from the transformed genres
contents = pd.DataFrame(genres, columns=vectorizer.get_feature_names_out())

# Print the shape of the content table
print('Shape of the content table:', contents.shape)

# Display the first few rows of the DataFrame
contents.head()

Shape of the content table: (995655, 20)


,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,film,horror,musical,mystery,noir,romance,sci,thriller,war,western
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(contents)

NearestNeighbors(metric='cosine')

In [51]:
class Recommender:
    def __init__(self):
        self.hist = [] 
        self.ishist = False

    def recommend_on_movie(self, movie, n_recommend=10):
        """
        Recommend movies similar to a given movie.

        Parameters:
            movie (str): The title of the movie.
            n_recommend (int): Number of recommendations to return.

        Returns:
            list: A list of recommended movie titles.
        """
        self.ishist = True
        iloc = content_merged_df[content_merged_df['Title'] == movie].index[0]
        self.hist.append(iloc)
        distance, neighbors = nn_algo.kneighbors([contents.iloc[iloc]], n_neighbors=n_recommend+1)
        recommeds = [content_merged_df.iloc[i]['Title'] for i in neighbors[0] if i not in [iloc]]
        return recommeds[:n_recommend]
    
    def recommend_on_history(self, n_recommend=10):
        """
        Recommend movies based on the user's watch history.

        Parameters:
            n_recommend (int): Number of recommendations to return.

        Returns:
            list: A list of recommended movie titles.
        """
        if not self.ishist:
            return print('No history found')
        
        unique_movies = set(content_merged_df['Title'])  # Create a set of unique movie titles
        
        history = np.array([list(contents.iloc[iloc]) for iloc in self.hist])
        distance, neighbors = nn_algo.kneighbors([np.average(history, axis=0)], n_neighbors=len(unique_movies))
        
        recommended_movies = []
        recommended_indices = set()  # To store indices of recommended movies
        
        for i in neighbors[0]:
            # Check if the movie is not in user's watch history, not already recommended, and is in the unique movie set
            if i not in self.hist and i not in recommended_indices and content_merged_df.iloc[i]['Title'] in unique_movies:
                recommended_movies.append(content_merged_df.iloc[i]['Title'])
                recommended_indices.add(i)
                if len(recommended_movies) == n_recommend:
                    break
                    
        # Filter out duplicates from recommended movies
        unique_recommended_movies = []
        for movie in recommended_movies:
            if movie not in unique_recommended_movies:
                unique_recommended_movies.append(movie)
        
        return unique_recommended_movies[:n_recommend]


In [52]:
def generate_user_watch_history(user_id, df):
    """
    Generate a particular user's watch history based on their user ID.

    Parameters:
        user_id (int): The ID of the user.
        df (pandas.DataFrame): The DataFrame containing the merged data.

    Returns:
        list: A list of movie titles representing the user's watch history.
    """
    user_history = df[df['UserID'] == user_id]['Title'].tolist()
    return user_history

# Example usage:
user_id = 1  # Replace with the desired user ID
watch_history = generate_user_watch_history(user_id, df)
print(f"User {user_id}'s watch history:")
print(watch_history)



User 1's watch history:
["One Flew Over the Cuckoo's Nest (1975)", 'James and the Giant Peach (1996)', 'My Fair Lady (1964)', 'Erin Brockovich (2000)', "A Bug's Life (1998)", 'The Princess Bride (1987)', 'Ben-Hur (1959)', 'A Christmas Story (1983)', 'Snow White and the Seven Dwarfs (1937)', 'The Wizard of Oz (1939)', 'Beauty and the Beast (1991)', 'Gigi (1958)', 'Miracle on 34th Street (1947)', "Ferris Bueller's Day Off (1986)", 'The Sound of Music (1965)', 'Airplane! (1980)', 'Tarzan (1999)', 'Bambi (1942)', 'Awakenings (1990)', 'Big (1988)', 'Pleasantville (1998)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'Back to the Future (1985)', "Schindler's List (1993)", 'Meet Joe Black (1998)', 'E.T. the Extra-Terrestrial (1982)', 'Titanic (1997)', 'Ponette (1996)', 'A Close Shave (1995)', 'Antz (1998)', 'Girl, Interrupted (1999)', 'The Hunchback of Notre Dame (1996)', 'The Hunchback of Notre Dame (1996)', 'The Hunchback of Notre Dame (1996)', 'The Hunchback of Notre Dame (199

In [55]:
def create_recommender_for_user(user_id, df):
    """
    Create a recommender for a specific user's watch history.

    Parameters:
        user_id (int): The ID of the user.
        df (pandas.DataFrame): The DataFrame containing the merged data.

    Returns:
        Recommender: A Recommender instance populated with the user's watch history.
    """
    user_watch_history = generate_user_watch_history(user_id, df)
    
    # Create a new Recommender instance
    user_recommender = Recommender()
    
    # Populate the Recommender instance with the user's watch history
    for movie_title in user_watch_history:
        user_recommender.recommend_on_movie(movie_title)
    
    return user_recommender

# Create a Recommender instance for user 1
user2_recommender = create_recommender_for_user(3, df)


# Now you have a Recommender instance for user 1 containing their watch history
# You can use this Recommender instance to make recommendations for user 1 based on their history


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not

In [56]:
user2_recommender.recommend_on_history()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['The Three Musketeers (1993)',
 'The Golden Child (1986)',
 'Operation Condor (1990)',
 'Mystery Men (1999)']

In [88]:
#old tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'overview' and 'tagline' are your textual columns
content_merged_df['combined_text'] = content_merged_df['overview'].fillna('') + ' ' + content_merged_df['tagline'].fillna('') + ' ' + content_merged_df['Title']

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(content_merged_df['combined_text'])




In [89]:
#old training
import pandas as pd

# Assuming df is your DataFrame and 'Timestamp' is the column with Unix timestamps
# Step 1: Convert 'Timestamp' to a datetime object
content_merged_df['Timestamp'] = pd.to_datetime(content_merged_df['Timestamp'], unit='s')

# Step 2: Sort the DataFrame by 'Timestamp' in ascending order
content_merged_df = content_merged_df.sort_values(by='Timestamp', ascending=True)

# Step 3: Determine the split point (e.g., 80% for training, 20% for test)
split_point = int(len(content_merged_df) * 0.8)

# Step 4: Split the dataset into training and test sets
train_set = content_merged_df[:split_point]
test_set = content_merged_df[split_point:]

# You now have your training set and test set, with the test set containing the most recent data
print(train_set)
print(test_set)

        Unnamed: 0  UserID  MovieID  Rating           Timestamp  \
995584      994914    6040      858       4 2000-04-25 23:05:32   
995319      994650    6040      593       5 2000-04-25 23:05:54   
995598      994928    6040     2384       4 2000-04-25 23:05:54   
995637      994967    6040     2019       5 2000-04-25 23:06:17   
995453      994783    6040     1961       4 2000-04-25 23:06:17   
...            ...     ...      ...     ...                 ...   
312909      312699    1875     1235       3 2000-12-02 14:34:55   
312644      312434    1875     3072       4 2000-12-02 14:34:55   
312817      312607    1875     3361       4 2000-12-02 14:35:12   
312818      312608    1875     3362       4 2000-12-02 14:35:12   
312615      312405    1875      909       3 2000-12-02 14:35:12   

                                  Title              Genres  \
995584             The Godfather (1972)  Action|Crime|Drama   
995319  The Silence of the Lambs (1991)      Drama|Thriller   
995598

In [93]:
#old
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def item_based_recom(input_dataframe, input_film_name): 
    pivot_item_based = pd.pivot_table(input_dataframe, index='Title', columns=['UserID'], values='Rating')
    sparse_pivot = sparse.csr_matrix(pivot_item_based.fillna(0)) 
    recommender = cosine_similarity(sparse_pivot) 
    recommender_df = pd.DataFrame(recommender, columns=pivot_item_based.index, index=pivot_item_based.index)
    cosine_df = pd.DataFrame(recommender_df[input_film_name].sort_values(ascending=False)) 
    cosine_df.reset_index(level=0, inplace=True) 
    cosine_df.columns = ['Title', 'cosine_sim']
    return cosine_df

def item_and_genre_based_recom(cosine_df, movies_df, categories):
    top_cos_genre = pd.merge(cosine_df, movies_df, on='Title')
    top_cos_genre['genre_similarity'] = [pairwise_row_diff(top_cos_genre, 0, row, categories) for row in top_cos_genre.index.values]
    return top_cos_genre[['Title', 'cosine_sim', 'genre_similarity']]

def pairwise_row_diff(dataframe, row1, row2, column_names):
    matrix_row1 = [[dataframe.loc[row1, cat] for cat in column_names.split('|')]] 
    matrix_row2 = [[dataframe.loc[row2, cat] for cat in column_names.split('|')]] 
    return round(cosine_similarity(matrix_row1, matrix_row2)[0][0], 5)

def generate_recommendations(df, UserID, top_results=10, cat=None): 
    # Get the top movie based on the rating of the UserID
    top_movie = df[df['UserID'] == UserID].sort_values(by='Rating', ascending=False)['Title'].iloc[0]
    
    print("Movie Recommender by Karan Walanj: ")
    print("User name: " + "Favorite Movie:", top_movie+'\n\n')
    print("Films you might enjoy based on what user", UserID, "watched:", top_movie)
    cos_sim = item_based_recom(df, top_movie) 
    display(cos_sim[1:top_results+1])

    print("Films you might enjoy with similar genre as", top_movie)
    display(item_and_genre_based_recom(item_based_recom(df, top_movie), df[['Title']], cat)
            .sort_values('cosine_sim', ascending=False)[top_results:]
            .sort_values('genre_similarity', ascending=False)[:top_results])
    return None

# Example usage
generate_recommendations(content_merged_df, UserID=25, top_results=10, cat=content_merged_df['Genres'])

Movie Recommender by Karan Walanj: 
User name: Favorite Movie: The Hunt for Red October (1990)


Films you might enjoy based on what user 25 watched: The Hunt for Red October (1990)


,Title,cosine_sim
1,The Fugitive (1993),0.697511
2,Patriot Games (1992),0.658445
3,Speed (1994),0.635091
4,Total Recall (1990),0.626389
5,Die Hard (1988),0.614651
6,The Rock (1996),0.612756
7,Terminator 2: Judgment Day (1991),0.611300
8,Clear and Present Danger (1994),0.611010
9,Die Hard 2 (1990),0.603285
10,The Matrix (1999),0.602715


Films you might enjoy with similar genre as The Hunt for Red October (1990)


AttributeError: 'Series' object has no attribute 'split'

In [17]:
#old
# #CONTENT BASED FILTERING 
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Assuming merged_df is already loaded with your data

# # Step 1: Preprocess the dataset with TF-IDF
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# merged_df['combined_features'] = merged_df[['Genres', 'spoken_languages', 'production_countries', 'production_companies', 'tagline', 'overview']].apply(lambda x: ' '.join(x.astype(str).values), axis=1)
# tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['combined_features'])

# # Function to create a user profile
# # def create_user_profile(user_id, merged_df, tfidf_matrix, rating_threshold=4):
#     user_data = merged_df[merged_df['UserID'] == user_id]
#     # Filter movies that the user has rated above the threshold
#     high_rating_indices = user_data[user_data['Rating'] > rating_threshold].index.tolist()
    
#     if not high_rating_indices:
#         return np.array([])  # Return an empty array if no high-rated movies

#     # We extract the rows from tfidf_matrix corresponding to high ratings
#     # and compute the mean. Note: toarray() converts sparse matrix to dense
#     # Ensure the result is a dense 2D array for compatibility with cosine_similarity
#     user_profile = np.mean(tfidf_matrix[high_rating_indices].toarray(), axis=0)
    
#     # Ensure user_profile is 2D: (1, number_of_features)
#     user_profile = user_profile.reshape(1, -1)
    
#     return user_profile


# # Function to recommend movies based on the user profile
# def recommend(user_profile, tfidf_matrix, merged_df, user_id, n_recommendations=5):
#     if user_profile is None:
#         return []

#     # Calculate similarity between user profile and all movie profiles
#     cosine_similarities = cosine_similarity(user_profile, tfidf_matrix)
    
#     # Get indices sorted by similarity (descending)
#     similar_indices = cosine_similarities.argsort().flatten()[-n_recommendations*2:]
    
#     # Filter out movies the user has already watched/rated
#     watched_movie_ids = set(merged_df[merged_df['UserID'] == user_id]['MovieID'])
#     recommended_movie_ids = [idx for idx in similar_indices if merged_df.iloc[idx]['MovieID'] not in watched_movie_ids]
    
#     # Limit to the top N recommendations
#     recommended_movie_ids = recommended_movie_ids[-n_recommendations:]
#     return merged_df.iloc[recommended_movie_ids]['MovieID'].values

# # Example usage
# user_id = 1  # Replace with an actual user ID
# user_profile = create_user_profile(user_id, merged_df, tfidf_matrix)
# recommended_movies = recommend(user_profile, tfidf_matrix, merged_df, user_id)
# print("Recommended movies for user {}: {}".format(user_id, recommended_movies))


KeyboardInterrupt: 